In [15]:
import pandas as pd
import random

1. Genere una muestra de casos (1 punto). Para ello debesconsiderar:
-  Utilice los últimos 4 dígitos de su rut como semilla pseudoaleatoria.
-  Seleccione el 50% de los casos.
-  Cada base generada debe contener los siguientes elementos:
   -  El índice de desarrollo humano (undp_hdi).
   -  El nombre del país (ccodealp).
   -  La región a la que pertenece (ht_region).
   -  El PIB per cápita. (gle_cgdpc).
   -  El total de la población (imf_pop)

- Si su apellido está entre la N y la Z, escoja las siguientes variables del módulo
Salud:
    -  wef_imort: Infant mortality, deaths/1000 live births.
    -  who_alc2000: Alcohol consumption per capita (2000-).
    -  who_tobt: Current smoking of any tobacco product (Total).
    -  wdi_exph: Government expenditure on health, total(% of GDP).
    -  Guarde esta tabla procesada en un nuevo objeto.
    -  Renombre las categorías de la variableht_region denúmeros a regiones.

In [53]:
def obtener_mitad_casos_df(df_largo):
    # Primero seria obtener el rango de indexes del df
    # Después sería obtener cuantos elementos se requieren
    # Después sería obtener la cantidad de elementos, pero sin repetirse
    cincuenta_porciento_casos = int(round(df_largo / 2, 0))
    indices_a_elegir = (random.sample(
        range(df_largo), cincuenta_porciento_casos))
    return indices_a_elegir


def obtener_submuestra(df):
    SEMILLA_PSEUDOALEATORIA = '0115'
    random.seed(SEMILLA_PSEUDOALEATORIA)

    columnas_a_utilizar = ['undp_hdi', 'ccodealp', 'ht_region', 'gle_cgdpc',
                           'imf_pop', 'wef_imort', 'who_alc2000', 'who_tobt', 'wdi_exph']

    casos_a_obtener = obtener_mitad_casos_df(df.shape[0])
    df_cincuenta_casos = df.iloc[casos_a_obtener]
    df_cincuenta_columnas_a_utilizar = df_cincuenta_casos[columnas_a_utilizar]

    return df_cincuenta_columnas_a_utilizar


In [54]:
df = pd.read_csv('qog_std_cs_jan18.csv')
df_auxiliar = pd.read_csv('subsample_isz.csv')

In [58]:
df_submuestra = obtener_submuestra(df)

In [59]:
df_submuestra

,undp_hdi,ccodealp,ht_region,gle_cgdpc,imf_pop,wef_imort,who_alc2000,who_tobt,wdi_exph
16,0.604,BTN,8,5639.02000,0.766,35.700001,0.23,NaN,3.573015
118,0.637,NAM,4,4521.29000,NaN,28.299999,7.84,23.799999,8.928688
60,0.470,DJI,4,2685.50000,NaN,NaN,0.38,NaN,10.568256
37,NaN,TWN,6,33965.87100,23.434,3.934564,NaN,NaN,NaN
11,0.823,BHR,3,22498.48000,1.268,8.200000,1.57,30.299999,4.980796
...,...,...,...,...,...,...,...,...,...
176,0.723,TUN,3,6912.27980,NaN,13.800000,1.39,NaN,7.003858
89,NaN,PRK,6,1727.81990,NaN,NaN,3.35,NaN,NaN
114,0.804,MNE,1,14538.52000,0.622,5.500000,6.56,NaN,6.416543
75,0.919,ISL,5,31313.60900,0.329,1.800000,7.45,17.500000,8.859166
